# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [6]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [11]:
#agrupo a los compradores y los productos por la cantidad que ha comprado dichos productos
df=data.groupby(['CustomerID', 'ProductName'])["Quantity"].sum().reset_index()
df.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [12]:
#pivot table para ver bien cuantos(cantidad) productos compra cada cliente
pivot = pd.pivot_table(df, values='Quantity', index=['ProductName'], columns=['CustomerID'], aggfunc=np.sum)
pivot=pivot.fillna(0)
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [20]:
#con esto vamos lo lejos o cerca que esta un cliente de otro por el tipo y numero de productos que ha comprado. 
#Aquí está la similaridad
y = squareform(pdist(pivot.T, 'euclidean'))
y

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [23]:
#para poder evaluarla mejor se hace 1/(1 + y) y así siempre queda la similaridad entr cero y uno.
distances = pd.DataFrame(1/(1 + y), index=pivot.columns, columns=pivot.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [101]:
#elijo uno cliente al azar y veo los 5 que están mas similares. realmente se ordenan y se hace un head de 5.
similarities = list(distances[639].sort_values(ascending=False)[1:].head(5).index)
similarities

[3317, 3535, 3074, 3305, 649]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [102]:
#Como ya tengo los clientes similares voy a por sus productos
#similar_ids=df[df["CustomerID"]==3317 & 3535 & 3074 & 3305 & 649]
similar_ids=df[df["CustomerID"].isin(similarities)]

similar_ids.head()


,CustomerID,ProductName,Quantity
510,649,Arizona - Green Tea,1
511,649,Bacardi Breezer - Tropical,1
512,649,Beans - Kidney White,1
513,649,"Beans - Kidney, Red Dry",1
514,649,"Beef - Chuck, Boneless",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [26]:
#como ya tengo todos los productos de cada cliente similar voy a agrupar y buscar los productos mas comprados por ellos
tops=similar_ids.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
tops.head()

,Quantity
ProductName,
Cod - Black Whole Fillet,4
Wine - Ej Gallo Sierra Valley,4
Yoghurt Tubes,3
"Pepper - Black, Whole",3
Halibut - Steaks,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [32]:
#vuelvo a la matriz original y le digo, de mi cliente cogido al azar, que me de todo lo que no haya comprado
nyp=pivot[pivot[639]==0]
# mergeo los productos que no he comprado con los mas comprados con los clientes mas similares a mi y cojo el top 5 
merged=tops.merge(nyp, left_on='ProductName', right_on='ProductName').sort_values("Quantity", ascending=False)
#me quedo con la columna del cliente y la cantidad de prodcutos, sino me sale todos los cientes.
merged[[639, "Quantity"]].head()

,639,Quantity
ProductName,,
Cod - Black Whole Fillet,0.0,4
Wine - Ej Gallo Sierra Valley,0.0,4
Yoghurt Tubes,0.0,3
"Pepper - Black, Whole",0.0,3
Soupfoamcont12oz 112con,0.0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [155]:
gpr={}
lst=data.CustomerID.unique().tolist()
for i in lst:
    similarities = list(distances[i].sort_values(ascending=False)[1:].head(5).index)
    similar_ids=df[df["CustomerID"].isin(similarities)]
    tops=similar_ids.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
    nyp=pivot[pivot[i]==0]
    merged=tops.merge(nyp, left_on='ProductName', right_on='ProductName').sort_values("Quantity", ascending=False)
    prueba=merged[[i,"Quantity"]][0:5].index
    gpr.update({i:prueba})


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [156]:
df_=pd.DataFrame(gpr)
df_

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,Jagermeister,Wine - Valpolicella Masi,Bread - Italian Corn Meal Poly,Squid - Tubes / Tenticles 10/20,Vanilla Beans,Cocoa Butter,Juice - Lime,Bread - French Baquette,Soup Knorr Chili With Beans,Cheese - Mozzarella,...,Beef - Top Sirloin,"Pasta - Penne, Rigate, Dry",Milk - 2%,Cheese - Mix,Chips Potato Salt Vinegar 43g,"Pasta - Penne, Rigate, Dry",Veal - Inside,Olives - Kalamata,"Veal - Brisket, Provimi,bnls","Pepper - Black, Whole"
1,Chicken - Soup Base,Tahini Paste,Puree - Mocha,Wanton Wrap,Lettuce - Frisee,"Cheese - Boursin, Garlic / Herbs",Pomello,Bandage - Flexible Neon,Soupfoamcont12oz 112con,Quiche Assorted,...,Grouper - Fresh,Soup Knorr Chili With Beans,Bay Leaf,Muffin Mix - Blueberry,Fenngreek Seed,Browning Caramel Glace,Wonton Wrappers,Wine - White Cab Sauv.on,Beef - Ground Medium,Butter - Unsalted
2,Lime Cordial - Roses,Tuna - Salad Premix,Tuna - Salad Premix,Puree - Mocha,"Yogurt - Blueberry, 175 Gr",Garlic - Peeled,Tomatoes Tear Drop,"Lentils - Red, Dry",Wine - Crozes Hermitage E.,"Lemonade - Natural, 591 Ml",...,Soup Knorr Chili With Beans,Coffee - Hazelnut Cream,"Veal - Inside, Choice",Cake - Box Window 10x10x2.5,Isomalt,Tea - Decaf Lipton,"Veal - Inside, Choice",Veal - Eye Of Round,"Rosemary - Primerba, Paste",Wine - Ej Gallo Sierra Valley
3,Macaroons - Two Bite Choc,Chicken - Soup Base,Sherry - Dry,Pernod,Cinnamon Buns Sticky,Cheese - Mix,Skirt - 29 Foot,Thermometer Digital,Beans - Kidney White,Bananas,...,Eggplant - Asian,Cocoa Butter,Banana Turning,Rum - Mount Gay Eclipes,Bay Leaf,"Garlic - Primerba, Paste",Bread Crumbs - Japanese Style,Bread - Calabrese Baguette,Scallops - 10/20,Wine - Blue Nun Qualitatswein
4,Flavouring - Orange,Isomalt,Beer - Sleemans Cream Ale,Sprouts - Baby Pea Tendrils,Pop Shoppe Cream Soda,Sauce - Hollandaise,Wine - Blue Nun Qualitatswein,Wine - Hardys Bankside Shiraz,Otomegusa Dashi Konbu,Wine - Chardonnay South,...,Lime Cordial - Roses,Rambutan,Wine - Vidal Icewine Magnotta,Scallops 60/80 Iqf,Veal - Sweetbread,"Wine - Red, Harrow Estates, Cab",Wasabi Powder,Cheese - Wine,"Oregano - Dry, Rubbed",Beef Wellington


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.